In [32]:
from IPython.display import Markdown, display
import PyPDF2
import requests
import os
from PyPDF2 import PdfReader
from openai import OpenAI
from dotenv import load_dotenv

In [33]:
def load_api_key():
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")

    # Check the key

    if not api_key:
        print(
            "No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!"
        )
    elif not api_key.startswith("sk-proj-"):
        print(
            "An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook"
        )
    elif api_key.strip() != api_key:
        print(
            "An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook"
        )
    else:
        print("API key found and looks good so far!")
    return api_key

In [34]:
api_key = load_api_key()
openai = OpenAI()

API key found and looks good so far!


In [35]:
BASE_PATH = "../data/crawled_content"
UNIVERSITY_STR = "uiw_edu" #  "stmary_edu" "klesse_utsa_edu"
full_path = f"{BASE_PATH}/{UNIVERSITY_STR}.txt"

In [36]:
# path = "C:\\Users\\alire\\OneDrive\\personal_projects\\mma_ranking\\Notebooks\\crawled_content_stmary_edu_6.txt"
# path = "C:\\Users\\alire\\OneDrive\\personal_projects\\mma_ranking\\Notebooks\\crawled_content_uiw_edu_6.txt"
# path = r"C:\Users\alire\OneDrive\personal_projects\mma_ranking\Notebooks\crawled_content_klesse_utsa_edu_1.txt"
print(f"Attempting to open: {full_path}")

with open(full_path, 'r', encoding='utf-8', errors='ignore') as file:
    content = file.read()    
    # print(content)

content = content.replace("|", "-")

Attempting to open: ../data/crawled_content/uiw_edu.txt


In [37]:
# OLLAMA_API = "http://localhost:11434/api/chat"
# HEADERS = {"Content-Type": "application/json"}
# MODEL = "llama3.2"


# def chat_with_ollama(prompt):
#     messages = [{"role": "user", "content": prompt}]
#     payload = {"model": MODEL, "messages": messages, "stream": False}
#     response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
#     return response.json()["message"]["content"]

In [38]:
system_prompt = """
You are a highly skilled information extraction assistant specializing in extracting structured data from unstructured web content. Your goal is to extract accurate and detailed information about university faculty and staff while following these guidelines:

1. Prioritize information from an individual's dedicated profile page. If data exists in multiple sources, trust the dedicated page for accuracy.
2. Use department directories only to supplement missing information or confirm the department name.
3. Ensure no hallucinations; extract data verbatim from the content.
4. Include all possible profiles, even if some fields are incomplete. Leave empty fields blank instead of excluding the record.
5. Extract all publications and courses exactly as listed on the pages, word by word, without summarization or modification.

The extracted data should be structured into the following fields:
- Name (Full name of the individual)
- Title (Academic or professional title)
- Office Location (Room and building details)
- Phone (Contact number)
- Email (Official university email address)
- Major (Field of study or specialization)
- Department (Academic department)
- Research Interests (Explicitly mentioned research areas)
- Teaching Interests (Courses taught or explicitly mentioned teaching areas)
- Publications (Each publication listed verbatim)
- Courses (Each course listed verbatim, including titles, codes, or descriptions)
- Page Link (The URL of the source webpage)

Output the extracted information in a structured table format, ensuring no entries are missed, even if incomplete.
"""

In [39]:
user_prompt = f"""
Extract the information for all faculty and staff mentioned in the given university webpages. Ensure that:

1. Data from an individual's dedicated page is prioritized over department directories for accuracy.
2. Department directories are used only to fill in missing information, such as the department name.
3. All publications and courses are included verbatim, as listed.
4. Incomplete profiles are still included, leaving NA where data is unavailable.

Provide the extracted data in a table format with the following headers:
| Name | Title | Office Location | Phone | Email | Major | Department | Research Interests | Teaching Interests | Publications | Courses | Page Link |

Input content:
{content}

"""

In [40]:
# print(user_prompt)

In [41]:
def chat_with_gpt(message, system_prompt=system_prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": message}
            ]
    )
    return response.choices[0].message.content

In [42]:
verbs_list = chat_with_gpt(user_prompt)
display(Markdown(verbs_list))

| Name                         | Title                                                 | Office Location                | Phone          | Email | Major                                      | Department                           | Research Interests                                            | Teaching Interests                                                                                              | Publications                                                                                                                                                                                                                                                               | Courses                                                                                                                                                                                                                                                                                                                                                                                                                                                   | Page Link                                                                                                        |
|------------------------------|-------------------------------------------------------|-------------------------------|----------------|-------|--------------------------------------------|--------------------------------------|-------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------|
| Jeannie J. Scott, Ph.D.      | Dean                                                  | GB 134 - Gorman Building      | (210) 283-5002 |       |                                            | H-E-B School of Business Administration |                                                            |                                                                                                                 | - How to Prepare Students to Compete in a Global Market, Jeannie J. Scott, Ph.D., AACSB International, Associate Deans Conference, Proceedings, Kansas City, MO, November 2015. <br>- Experienced and Potential Medical Tourists’ Service Quality Expectations. Michael Guiry, Ph.D., Jeannie J. Scott, Ph.D., David G. Vequist, Ph.D., International Journal of Health Care Quality Assurance, Vol. 26, No. 5, pp. 433-446, 2013. <br>- Coach, Mentor, or Supervisor: Which are you? Jeannie J. Scott, Ph.D., Southern Association of Collegiate Registrars and Admissions Officers 66th Annual Meeting, Conference Proceedings, San Antonio, TX, February 2013. <br>- Internship Management, Jeannie J. Scott, Ph.D., Academy of Business Research, proceedings, Atlantic City, N.J., 2012. <br>- Study of Domestic Medical Tourism, Shishu Zhang, Ph.D., David Fike, Ph.D., Jeannie J. Scott, Ph.D., The Southern Business and Economic Journal, 2012. <br>- Locus of Internship Management: Does it Matter?, Henry Elrod, Ph.D., Jeannie J. Scott, Ph.D., Theresa Tiggeman, CPA, Journal of Case Studies in Accreditation and Assessment, 2012. <br>- Locus of Internship Management: Does it Matter?, Henry Elrod, Ph.D., Jeannie J. Scott, Ph.D., Theresa Tiggeman, CPA, Academic & Business Research Institute International Conference Proceedings, San Antonio, TX, March 2012. | Graduate and Undergraduate Courses:<br>- Analytical Decision Making I<br>- Analytical Decision Making II<br>- Analytical Decision Making III<br>- Business Statistics<br>- Management Science<br>- Integrative Business Analysis and Decision Making I (Capstone I)<br>- Integrative Business Analysis and Decision Making II (Capstone II)<br>- International Policies and Relations<br>- Business Research and Analysis<br>- Quantitative Decision Making in Business<br>- MBA Capstone<br>- Internships-All Disciplines<br>- Leadership for Successful Entrepreneurs<br>- Theoretical Foundations in Business | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/scott-jeannie.html)  |
| Taylor Collins, Ph.D.        | Department Chair of Economics/Management/General Business | GB 206 - Gorman Building      | (210) 829-3187 |       | Economics                                  | H-E-B School of Business Administration | Macroeconomics, Energy Economics, Time Series Econometrics  | - Introduction to Macroeconomics<br>- Introduction to Microeconomics<br>- Public Finance<br>- Econometrics<br>- Macroeconomic Forecasting<br>- Managerial Economics    |                                                                                                                                                                                                                                               | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/collins-taylor.html)  |
| Haley Ayres, M.Ed.           | Advising Specialist                                   | GB 105 - Gorman Building      | (210) 930-8014 |       |                                            | H-E-B School of Business Administration |                                                            |                                                                                                                 |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/ayres-haley.html)    |
| Joshua Bazzy, Ph.D.         | Management                                            | AGO 109 - Agnese Sosa Building| (210) 283-6347 |       |                                            | H-E-B School of Business Administration | Integrity, Counterproductive Work Behavior, Self-Control, Entrepreneurship | - Human Resource Management<br>- Management Science<br>- Management Theory and Practice<br>- Business Statistics<br>- Integrative Business Analysis and Decision Making  | - Bazzy, J. D., Woehr, D. J., & Borns, J. (2017). An examination of the role of self-control and impact of ego depletion on integrity testing. Basic and Applied Social Psychology, 39 (2), 101-111. <br>- Bazzy, J. D. & Woehr, D. J. (2017). Integrity, ego depletion, and the interactive impact on counterproductive behavior. Personality and Individual Differences, 105, 124-128. <br>- Bazzy, J. D. (2016). The role of abstract thinking in entrepreneurial intentions and actions. Academy of Management Annual Meeting, Anaheim, California.<br>- Bazzy, J. D. (2016). Work ethic dimensions as predictors of ego depletion. Current Psychology, in press.  |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/bazzy-joshua.html)    |
| Carlos Baldo, Ph.D.         | International Business                                | GB 222 - Gorman Building      | (210) 805-2538 |       |                                            | H-E-B School of Business Administration | Executive Recruitment, Emerging Markets, Agency Theory        | - International Business Management<br>- International Marketing                                          |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/carlos-baldo-ph.d.html) |
| Lynn T. Downs, Ph.D.        | Health Administration                                  | AGO 112 - Agnese Sosa Building| (210) 829-3177 |       |                                            | H-E-B School of Business Administration | Leadership Succession Planning, Competency-Based Education, Moral Hazard in Healthcare | - Leadership<br>- Organizational Behavior and Theory<br>- Human Resources<br>- Health Policy                      | - Richter, J., Downs, L., Beauvais, B., The Impact of Open Access Appointing on Patient Satisfaction; Quality Management in Healthcare, 26(1): 22-28, January/March 2017 <br>- Beauvais, B., Richter, J., Kim, F., Weigel, F., Brezinski, P., Downs, L., ... & Powers, J. (2018). The Influence of CAHME Accreditation Findings and Program Reputation on Length of Healthcare Management Program Accreditation. Journal of Health Administration Education, 34(4), 569-584.<br>- Richer, J., Beauvais, B., Downs. L., Kim, F., Staff Perceptions of Organizational Patient Safety Climate and Quality Outcomes: An Examination of Outpatient Medical Offices, Medical Care and Research Review, Submitted August 2018. |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/downs-lynn.html)          |
| Trish Driskill, Ph.D.       | Professor                                             | AGO 109 - Agnese Sosa Building| (210) 829-3184 |       | Accounting                                  | H-E-B School of Business Administration | Accounting Ethics, Accounting Scandals, Forensic Accounting | - Accounting Ethics<br>- International Accounting<br>- Audit<br>- Principles of Accounting I<br>- Principles of Accounting II                           | - Driskill, T. G. (2016). Real-world ethical dilemmas in introductory accounting (Doctoral dissertation). Retrieved from Proquest Dissertations and Theses. <br>- Driskill, T. G. (2016). Quasi-experimental investigation of ethical dilemma quandaries in introductory accounting. Manuscript in preparation. Target journal: Issues in Accounting Education. <br>- Driskill, T. and Tiggeman, T. (2017). Guanxi: The paradigm of Chinese culture and Western perspective. Allied Business Academies (ABA) 2017 International Conference. |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/driskill-trish.html)  |
| Tracie Edmond, Ph.D.        | Accounting                                           | GB 106 - Gorman Building      | (210) 930-8016 |       |                                            | H-E-B School of Business Administration | Service Learning, Tri-sector Collaboration, Taxes         | - Accounting I<br>- Accounting II<br>- Cost Accounting<br>- Non-Profit Accounting<br>- Managerial Accounting                                           | - Book: Tri-sector Collaborations, The Benefits of Three Sectors Working Together, Scholars Press, 2015 <br>- Presentation: Utilizing Tri-Sector Collaboration in Service-Learning Projects, Edmond, 2015                                                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/edmond-tracie.html)  |
| Michael P. Forrest, J.D.    | Business Law                                         | GB 103 - Gorman Building      | (210) 930-8016 |       |                                            | H-E-B School of Business Administration | International Law, Accounting Law                               | - International Business Law<br>- Accounting Law                                                                     | - Forrest, M., & Norris, J.T. (2013). Bribery and China go together like yin and yang. Cumberland Law Review, 44(3), 423-432.<br>- Elrod, H. & Forrest, M. (2014). Homeownership is not always a blessing. Journal of the International Academy for Case Studies. Allied Academies International Internet Conference Proceedings, 15(2), 71-74.                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/forrest-michael.html) |
| Akbar Ghiasi, Ph.D.        | Health Administration                                | AGO 105 - Agnese Sosa Building| (210) 283-6484 |       |                                            | H-E-B School of Business Administration | Strategic Management in Healthcare, Long-term Care, Hospital Performance | - Population Health and Epidemiology<br>- Quantitative Analysis for Healthcare Managers<br>- Research Methods and Quantitative Analysis<br>- Business Statistics |                                                                                                                                                                                                                                                                                                                                                                           |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/ghiasi-akbar.html)   |
| Tim Griesdorn, Ph.D.         | Finance                                             | AGO 111 - Agnese Sosa Building| (210) 283-6332 |       |                                            | H-E-B School of Business Administration | Behavioral Finance, Wealth Creation, Risk Tolerance             | - Principles of Financial Management<br>- Risk Management and Insurance Planning<br>- Retirement Planning<br>- Financial Management (Graduate)<br>- Principles of Financial Planning | - Griesdorn, T. S., Durband, D. B. (2015). Does self-control predict wealth creation among baby boomers? Journal of Family and Economic Issues, 37(1), pg. 18-28.<br>- Sages, R. A., Griesdorn, T. S., Gudmunson, C. G., and Archuleta, K. L. (2014). Assessment in Financial Therapy. In B. Klontz, S. Britt, and K. Archuleta, (Eds.), Financial Therapy (pp 69-85). New York, NY: Springer.<br>- Smith, H. L. and Griesdorn, T. S. (2014). Small business and self-employed retirement account participation. Family and Consumer Sciences Research Journal, 43(1), 47-60.   |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/griesdorn-tim.html)     |
| Randall Griffiths, Ph.D.    | Sport Management                                     | WC 130 - Wellness Center       | (210) 829-2795 |       |                                            | H-E-B School of Business Administration |                                                            |                                                                                                                 |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/griffiths-randall.html)  |
| Earl Harmsen, MBA           | Management                                           | GB 220 - Gorman Building       | (210) 283-5003 |       |                                            | H-E-B School of Business Administration |                                                    |                                                                                                                 |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/harmsen-earl.html)   |
| Teresa Harrison, Ph.D.      | Management                                           | GB 207 - Gorman Building       | (210) 930-8011 |       |                                            | H-E-B School of Business Administration |                                                  |                                                                                                                 | - Harrison, D., Harrison, T., & Shaffer, M. (2021; in press). HRM challenges for immigrant employees: Status-laden transitions across cultures and workplace social environments. In M.R. Buckley, A.R. Wheeler, J.E. Bauer, & J.R.B. Halbesleben (Eds.), Research in Personnel and Human Resource Management, 39, 205-239. London: Elsevier.<br>- Harrison, D., Harrison, T., and Shaffer, M. (2019). Strangers in strained lands: Workplace experiences of immigrant employees. Journal of Management, 45 (2), 600-619. | - BMGT 6399 Special Topics: Leading a Diverse Workforce (Graduate)<br>- BMGT 6399 Special Topics: Talent Management (Graduate)<br>- BMGT 6311 Managing People and Organizations (Graduate)<br>- BMGT 6311 Human Resource Management (Graduate)<br>- BMGT 4380 Capstone I: Integrative Business Analysis and Decision Making (Undergraduate)<br>- BMGT 4355 Organizational Behavior (Undergraduate)<br>- BMGT 3380 Business Communication (Undergraduate)<br>- BMGT 3370 Business Statistics (Undergraduate) | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/harrison-teresa.html) |
| Alan A. Jones, PhD          | Health Administration                                   | AGO 105 - Agnese Sosa Building| (210) 829-3181 |       |                                            | H-E-B School of Business Administration | Health Insurance, Patient Satisfaction/Patient Experience, Comparative Health Systems | - HADM 6302-Healthcare Economics<br>- HADM 6360-Managing Healthcare Organizations<br>- HADM 6380-Healthcare Policy<br>- HADM 63CS-Healthcare Administration Capstone Course |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/jones-alan.html)  |
| Angelina Kiser, Ph.D.       | Management                                            | GB 107 - Gorman Building       | (210) 283-6484 |       |                                            | H-E-B School of Business Administration | Ethics, Cultural Diversity, Information Systems, Gender Issues | - Business Ethics<br>- Management Theory and Practice<br>- Seminar in Business and Society<br>- International Management Practices | - Rodriguez, A. & Kiser, A. I. T. (2015). Gender and age differences in servant leadership. Academy of Business Research Journal, 1, 49-63.<br>- Kiser, A. I. T. & Washington, R. (2015). The information gap among the generations and the implications for organizations. International Journal of Digital Literacy and Digital Competence, 6(2), 36-63.<br>- Kiser, A. I. T. (2015). Workplace perceptions between men and women. Gender in Management: An International Journal, 30(8), 598-612. |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/kiser-angelina.html)  |
| Yi Liu, Ph.D.               | Management Information Systems                         | AGO 107 - Agnese Sosa Building| (210) 832-5673 |       | Management Information Systems (MIS)     | H-E-B School of Business Administration | Platform-Centric Competition, Business Analytics, E-Business, IT Business Value and IT Strategy | - ISM 3003 Foundations of Management Information Systems<br>- MIS 2321 Introduction to Information Systems<br>- MIS 2325 Information Management Application<br>- MIS 3362 Systems Analysis & Design<br>- MIS 3325 Introduction to Business Analytics and Business Intelligence<br>- MIS 4325 Predictive Analysis and Forecasting<br>- MIS 4330 Data Mining and Data Analytics<br>- ACCT 6360 Accounting Data Analytics and Business Intelligence | - Liu, Y., Zhao, X., Liu, Y, Wang, J., Liu, Y., Li, P., Lin, Z., Sindakis, S., & Aggarwal, S. (2023). Overview of Data Quality.<br>- Liu, Y. (2015). The Effects of Big Data Analytics and Innovation Culture on Absorptive Capacity and Innovation. In SIGBPS Workshop on Business Processes and Services 2015 Processing Research Papers, Fort Worth, Texas.<br>- Armstrong, D. J., Liu, Y., and Riemenschneider, C. K. (2015). How Managers and Workers See Their World: Perceptions of the Relationship between Organizational Capabilities and Absorptive Capacity in US State Information Systems. In the European Conference on Information Systems Completed Research Papers, Munster, Germany. | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/liu-yi.html)  |
| Jose Moreno, Ph.D.           | Department Chair - Finance, Marketing & International Business | AGO 111 - Agnese Sosa Building| (210) 829-6018 |       | Finance                                    | H-E-B School of Business Administration | Market Microstructure, Finance Theory of Capital Markets, International Finance      | - Principles of Financial Management (Undergraduate)<br>- Financial Management (Graduate)<br>- Investments (Undergraduate and Graduate)<br>- International Economics (Undergraduate)<br>- Econometrics for Finance (Graduate) | - Moreno, J.F, Guiry, M. 2014. The relationship between Income Level and Medical Tourism Service Quality Expectations. International Journal of Services, Economics, and Management, Vol. 6(4), p. 263-281.<br>- Moreno, J.F., Torres, L., Jackson, S., 2014. The Study Abroad Experience in Enhancing Cross-Cultural Tolerance and Communication. Experiential Learning in International Business, (ed. Palgrave). September 2014.<br>- Moreno, J.F., Simpson, M.W., Ozuna, T. 2012. The Makings of an Information Leader: The Intraday Price Discovery Process for Individual Stocks in the DJIA. Review of Quantitative Finance and Accounting, Vol. 38 (3), p. 347-365. | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/moreno-jose.html)  |
| Chris Nesser, Ph.D.          | MHA Director                                         | GB 205 - Gorman Building      | (210) 832-5626 |       |                                            | H-E-B School of Business Administration | Health Policy, Access and Financing of Healthcare Delivery, Interprofessional Education and Practice | - Healthcare Finance I and II<br>- Healthcare Marketing<br>- Healthcare Policy<br>- Accounting Concepts | - Dominguez, D. G., Garcia, D.C., LaFrance, K. G., (2013). Developing a New Graduate Program in Healthcare Management: Embracing the Transformation of Healthcare Management Education on a Pathway to Success. Administrative Issues Journal: Education, Practice, and Research, V3(1).<br>- LaFrance, K., Garcia, D., and Dominguez, D. "The Concurrent Graduate Assistantship: Lessons from the Ground Up." Annual Conference of the Association of University Programs in Health Administration. June 3-6, 2015 Miami, FL. Peer Reviewed Poster Presentation. |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/nesser-chris.html)  |
| JT Norris, Jr., Ph.D.       | Professor                                            | GB 224 - Gorman Building      | (210) 829-3181 |       |                                            | H-E-B School of Business Administration | Ethics, China, Accounting Faculty Shortage                                 | - Advanced Financial Reporting<br>- Principles of Accounting I                                              | - Excel Training & the Technology Student Learning Outcome. Co-authored with Henry Elrod, Kelly Pittman, and Theresa Tiggeman. (in press, 2014). Academy of Business Research Journal, 6.<br>- Bribery and China Go Together Like Yin and Yang, Co-authored with Michael P. Forrest (Cumberland Law Review, Volume 44 (3) September 2014, Cumberland School of Law, Samford University.)       |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/norris-jt.html)   |
| Ivan Ortega                  | Administrative Assistant to the Dean               | GB 137 - Gorman Building      | (210) 283-5007 |       |                                            | H-E-B School of Business Administration |                                                            |                                                                                                                 |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/ortega-ivan.html)   |
| Kelly Pittman, M.S.         | Accounting                                           | GB 224 - Gorman Building      | (210) 829-6018 |       |                                            | H-E-B School of Business Administration | Technology in Accounting Courses, Business Sustainability             | - Managerial Accounting<br>- Intermediate Accounting I and II<br>- Auditing<br>- Advanced Financial Reporting                                      | - Pittman, K. and Moore, A. (2015). The Importance of Business Sustainability: Teaching Through Study Abroad. Center for Scholastic Inquiry- International Academic Research Conference. (Presentation). Charleston, South Carolina.<br>- Elrod, H., Pittman, K., Norris, J., and Tiggeman, T. Excel training & the technology student learning outcome. Journal of the Academy of Educational Leadership. |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/pittman-kelly.html)  |
| April Poe, Ph.D.            | Department Chair - Accounting, Business Law & MIS  | GB 221 - Gorman Building      | (210) 829-3181 |       |                                            | H-E-B School of Business Administration | Taxation, Financial Reporting, Regulation                              | - Taxation (Graduate and Undergraduate)<br>- Principles of Accounting                                      | - Comparison of Tax/Legal Research Services with T. Tiggeman and M. Forrest presented and published in the proceedings for the Allied Academics Winter International Conference, December 2014.<br>- Characteristics of Accounting Standards and SEC Review Comments with J. Boone and C. Linthicum (UTSA) Accounting Horizons, December 2013.                                     |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/poe-april.html)    |
| Robin Guerrero, Ph.D.       | Accounting                                           | GB 222 - Gorman Building      | (210) 829-6018 |       |                                            | H-E-B School of Business Administration |                                                            |                                                                                                                 |                                                                                                                                                                                                                                                                            |                                                                                                                                                                                                                                                                                                                                                                                                                                                   | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/robin-guerrero-ph.d.html)  |
| Alberto Rubio, Ph.D.        | MBA Director                                        | GB 225 - Gorman Building      | (210) 930-8766 |       |                                            | H-E-B School of Business Administration | Product Adoption, Servant Leadership, Cross-cultural behavior         | - Principles of Marketing<br>- Marketing Research<br>- Consumer Behavior<br>- Advanced Marketing Strategy                     | - Duarte, A. & Rubio-Sanchez, A. (2014). What can MLS do for you? Academy of Business Research Conference Proceedings. San Antonio, Texas. (Best Paper Award)<br>- Rubio-Sanchez, A. & De los Santos, E. (2014). Fair Trade Branding as a tool for the Marketing of Developing Society Crafts. Southwest Decision Sciences Institute Conference Proceedings. Dallas, Texas.<br>- Esteva-Armida, E. & Rubio-Sanchez, A. (in Print). The Influence of Trust in the UTAUT Model. In A. Allyson (Ed.), Analyzing Security, Trust, and Crime in the Digital World. Hershey, PA: IGI Global.           | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/rubio-alberto.html)  |
| Alicia Rubio, Ph.D.         | Undergraduate Director                               | GB 203 - Gorman Building      | (210) 930-8767 |       |                                            | H-E-B School of Business Administration | Minority household savings, Retirement planning, Servant Leadership                                         | - Principles of Financial Planning<br>- Principles of Financial Management<br>- Intermediate Financial Management                           | - Rubio, A. R. (2015). Factors Associated with Households Planning Horizons for Making Saving and Spending Decisions. Family and Consumer Sciences Research Journal, 43(3), 284-292.<br>- De Rubio, A. R. & Galvez-Kiser, A. (2015) Gender and Age Differences in Servant Leadership. Academy of Business Research Journal. Vol I. Pp. 49-63.                                        | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/rubio-alicia.html)  |
| Scott Roberts, Ph.D.        | Professor                                            | AGO 112 - Gorman Building      | (210) 283-5007 |       |                                            | H-E-B School of Business Administration | Consumers, Digital media, Authenticity, Branding           | - Advertising/ Integrated Marketing Communications<br>- Internet Marketing<br>- International Marketing                                         | - Scott D. Roberts and Kathleen S. Micken (2015) Marketing Digital Offerings Is Different: Strategies for Teaching About Digital Offerings in the Marketing Classroom, Journal of Education for Business, 90:2, 96-102, DOI: 10.1080/08832323.2014.988200.                                                  | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/scott-roberts,-ph.d..html) |
| David Vequist, Ph.D.        | Management                                           | GB 218 - Gorman Building      | (210) 805-5825 |       |                                            | H-E-B School of Business Administration | Medical Tourism, Health Tourism, Retirement Tourism, Social Media Predictive Analytics, Strategy, Human Resources | - Capstone<br>- Entrepreneurship                                                                      | - Vequist, D. G. (in press, 2020). The Hotel at a Hospital Strategy: An Analysis. International Journal of Business Strategy and Automation (IJBSA).<br>- Vequist, D. G. (2020). Medical Travel- NWIP. ISO TECHNICAL COMMITTEE 304 HEALTHCARE ORGANIZATION MANAGEMENT.<br>- Vequist, D. G., & Harmsen, E. (2018). A Logical Model for the Reducing of Healthcare Costs. UIW Research Week. | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/vequist-david.html)   |
| Nürşen Albayrak Zanca, Ph.D.| Economics                                           | GB 226 - Gorman Building      | (210) 829-3178 |       |                                            | H-E-B School of Business Administration | Economic Development, Economics and Peace, Meta Analysis, Economic Literacy | - Principles of Macroeconomics<br>- Principles of Microeconomics<br>- Managerial Economics<br>- Econometrics<br>- Economic Development<br>- Analyzing Economic Data<br>- Math for Economists<br>- Quantitative Business Methods           | - Zanca, N. A. (2016-17). “Peace Through Commerce and Peace Through Education” (under review) Palestine-Israel Journal of Politics, Economics and Culture.<br>- Zanca, N. A. (2016-17). “The Economic Value of Peace to the World: An Economist’s Perspective” (under review) The Exchange.<br>- Zanca, N. A. (2017). “Time as an Economic Resource: Scarce and Unequal” Paper Presentation at the 28th Economics Teaching Conference at Robert Morris University, February 2017.<br>- Zanca, N. A. (2016). “Putting Peace into UG Economics Curriculum” Journal of Business and Economics ISSN 2155-7950, Fall 2016.<br>- Zanca, N. A. (2015). “Lecture vs. Experiments in Economics Learning: A META Analysis from Journal of Economic Education (1969 to 2015)” Financial Education Association (FEA), 16th Annual Conference, Received Best Paper Award. | [Link](https://www.uiw.edu/hebsba/faculty-and-staff/directory/zanca-nursen-albayrak.html)  |


In [43]:
# verbs_list = chat_with_gpt(user_prompt)
# display(Markdown(verbs_list))

In [44]:
import pandas as pd
import re
# TODO: the functions does not consider empty cells. Should be fixed.
def save_as_csv(text, file_name="output/professors_list.csv"):

    # Extract the header and rows using regular expressions
    lines = text.splitlines()
    table_lines = [line for line in lines if "|" in line]  # Keep only table lines

    # Process the header and rows
    header = [col.strip() for col in table_lines[0].split("|")[1:-1]]  # Extract header
    data = [
        [col.strip() for col in row.split("|")[1:-1]]  # Extract each row
        for row in table_lines[2:]  # Skip separator row
    ]

    # Create a DataFrame
    df = pd.DataFrame(data, columns=header)

    # Save to CSV with UTF-8 BOM
    df.to_csv(file_name, index=False, encoding="utf-8-sig")

    print(f"CSV file saved as {file_name}' with UTF-8 BOM.")

In [45]:
# print(verbs_list)

In [46]:
save_as_csv(verbs_list, file_name=f"../data/professors_list/{UNIVERSITY_STR}.csv")

CSV file saved as ../data/professors_list/uiw_edu.csv' with UTF-8 BOM.
